In [69]:
!pip install openai==1.3.7

  Obtaining dependency information for openai==1.3.7 from https://files.pythonhosted.org/packages/3e/d3/309769dad11d5f75b81c7252d9dc849ed440d0921215e759af169054f3b6/openai-1.3.7-py3-none-any.whl.metadata
  Using cached openai-1.3.7-py3-none-any.whl.metadata (17 kB)
Using cached openai-1.3.7-py3-none-any.whl (221 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [120]:
from openai import OpenAI
import os

client = OpenAI(
    # api_key=
    api_key='api-key'
)

In [56]:
!pip install llmlingua


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [57]:
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [58]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('./lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [59]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [60]:
gsm8k_test = gsm8k['test']

In [61]:
prompt_complex = open('./lib_prompt/prompt_hardest.txt').read()

In [62]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [109]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [64]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [65]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [66]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [73]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Follow the given examples and answer the question."
        },
        {
            "role": "user", 
            "content": prompt_q
        }
    ],
    model="gpt-3.5-turbo",
)

In [86]:
print(response.choices[0].message.content)

The robe takes 2 bolts of blue fiber and half that much white fiber, which means the robe takes 2 + (1/2) * 2 = 3 bolts in total. 

The answer is 3 bolts.


LLMlingua2를 이용한 prompt compress 

In [102]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    use_llmlingua2=True, # Whether to use llmlingua-2
    device_map='cpu'
)
compressed_prompt = llm_lingua.compress_prompt(prompt_complex, rate=0.1)['compressed_prompt']

Token indices sequence length is longer than the specified maximum sequence length for this model (2209 > 512). Running this sequence through the model will result in indexing errors


In [103]:
print(compressed_prompt)

Angelo Melanie 2 chapters 4 worksheets 3 hours 1. 5 hours 4 hours 30 minutes lunch 3 hours 1. 5 hours 12 30 12 4 days Mark team 25 pointers 10 free throws points 25 pointersteam scores 6 pointers 8 10 scored 50 84 points double half scored 100 117 points 84 + 117 = 201 Bella two times marbles 20 frisbees 2 / 5 * 60 24 20deck cards 10 14 14 42 140 9 apples 15 oranges 14 bananas 9 + 24 3 114 9 - 2 7 15 13 + 20 14 - 12 20 32 4 36 $ 0 50watermelon W 4 1W = 4A 12 watermelons $ 60 $ 60 = 12W convert $ 60 one apple $ 1 Susy 800 300 100 40 third 90 100 140 40 10 30 180 180 $ 3 $profit bought 12 120 360 5 sold 15 330 2 220 earned 220 235 115 2 / 3 1 5 3 : 2 3 5 1000 200 3 = 600 males 400 480


2366 토큰이던 프롬프트를 298 토큰으로 압축

In [104]:
prompt_q_with_compressed_prompt = compressed_prompt + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'
response_with_compressed_prompt = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Follow the given examples and answer the question."
        },
        {
            "role": "user", 
            "content": prompt_q_with_compressed_prompt
        }
    ],
    model="gpt-3.5-turbo",
)

In [105]:
print(response_with_compressed_prompt.choices[0].message.content)

To find the total number of bolts required for the robe, we first calculate the amount of white fiber needed.

Given that the robe requires 2 bolts of blue fiber and half that much white fiber, it requires 2/2 = 1 bolt of white fiber.
Therefore, the total number of bolts needed for the robe is 2 (blue fiber) + 1 (white fiber) = 3 bolts. 

So, it takes a total of 3 bolts to make the robe.


평가 수행 - 원본 프롬프트

In [115]:
test_count = 50

In [116]:
i = 0
with open('./outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == test_count): break

  4%|▎         | 49/1319 [02:01<52:31,  2.48s/it]  


In [117]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')

num_q 50 correct 30 ratio 0.6000


평가 수행 - 압축 프롬프트

In [118]:
i = 0
with open('./outputs/test_gpt_3.5_turbo_complex_compressed.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_q_with_compressed_prompt + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == test_count): break

  4%|▎         | 49/1319 [03:00<1:17:49,  3.68s/it]


In [119]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_compressed.txt')

num_q 50 correct 22 ratio 0.4400


압축된 프롬프트로 수행 시 정확도가 떨어지는 것을 확인할 수 있었습니다.